In [1]:
import tensorflow as tf
import utils
import pandas as pd
import numpy as np
__author__ = "Olivares Castillo José Luis"

tf.__version__

'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
na_dummy = na.drop(na.columns[0],axis=1)
na_vectores1 = np.array(na_dummy)

In [4]:
eval_set = pd.read_csv("../lexiconevaluacion/evaluationset",delimiter=" ",names=["esp","nah"])

In [5]:
eval_set.head()

,esp,nah
0,traer,itqui
1,traer,huica
2,traer,huiqui
3,traer,cui
4,adornar,apana


In [6]:
eval_es = list(set(eval_set["esp"]))

In [7]:
eval_es

['ver',
 'efecto',
 'alfredo',
 'pipián',
 'enorme',
 'vaca',
 'nido',
 'niño',
 'conocer',
 'sólo',
 'caja',
 'vasija',
 'acabar',
 'fino',
 'final',
 'alegrar',
 'adornar',
 'coyote',
 'comer',
 'diverso',
 'adquisición',
 'verde',
 'llanura',
 'venir',
 'patio',
 'vender',
 'teatro',
 'echar',
 'muñeca',
 'tepaneca',
 'pieza',
 'indio',
 'derribar',
 'vestido',
 'pasar',
 'establecer',
 'tomar',
 'quitar',
 'chiquillo',
 'portal',
 'aguacate',
 'galleta',
 'jadear',
 'llevar',
 'barca',
 'puramente',
 'cultura',
 'seno',
 'ocurrir',
 'desierto',
 'ropa',
 'esfuerzo',
 'dar',
 'cargo',
 'tapar',
 'rogar',
 'nombre',
 'levantar',
 'tortuga',
 'alimento',
 'aire',
 'quetzal',
 'horno',
 'nabo',
 'descanso',
 'ciudad',
 'amargo',
 'animal',
 'muerto',
 'sueño',
 'destrozar',
 'pedirle',
 'mirada',
 'tigre',
 'ramo',
 'presidente',
 'familiar',
 'paso',
 'yo',
 'pez',
 'izquierda',
 'tráiganla',
 'presencia',
 'uña',
 'gritar',
 'volver',
 'indígena',
 'desear',
 'amado',
 'quiltoniles',

In [8]:
eval_es_index = [int(es[es[0] == palabra].index[0])
                  for palabra in eval_es]

In [9]:
es[es[0] == "pluma"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
2902,pluma,-0.386967000000000005,0.005026,-0.191887000000000002,-0.081058000000000005,-0.211675000000000002,-0.217542999999999986,-0.019026000000000001,0.454046000000000005,0.006309,...,-0.013390000000000001,0.023695999999999998,0.394602000000000008,-0.003221,-0.708667000000000047,0.021819000000000002,0.181774999999999992,0.436242000000000019,-0.162193000000000004,-0.076617000000000005


In [10]:
eval_es_index[0]

1183

In [11]:
eval_es_vectores = utils.get_vectors(es,eval_es_index)

In [12]:
eval_es_vectores.shape

(150, 128)

In [13]:
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in eval_es_index])

In [14]:
test_vectors[0]

array([-0.080436, -0.42478 , -0.147925, -0.189753,  0.22598 , -0.35795 ,
       -0.414704, -0.076678,  0.093425, -0.04006 , -0.042414, -0.245974,
        0.084349, -0.310707, -0.079203,  0.074513, -0.309028,  0.499327,
       -0.197057, -0.201877, -0.16073 , -0.073177,  0.116325,  0.035551,
        0.172715,  0.060754,  0.040407,  0.239737, -0.042063,  0.193151,
       -0.048046,  0.091137, -0.325254,  0.207736,  0.038816,  0.268411,
        0.374894,  0.095901,  0.169829,  0.329699,  0.381306, -0.262141,
       -0.240275,  0.420961, -0.271935, -0.168889,  0.085465, -0.009695,
       -0.216113, -0.083395, -0.01047 ,  0.248494, -0.475851, -0.18608 ,
        0.022355,  0.317714, -0.153873, -0.294219,  0.030675,  0.038219,
       -0.073873, -0.262666, -0.035965, -0.014698, -0.135102, -0.491196,
        0.005492, -0.199449,  0.1035  ,  0.430013,  0.219158,  0.354197,
        0.081869, -0.049073,  0.004599,  0.026837,  0.146431, -0.050878,
        0.11629 ,  0.216242, -0.095904, -0.095062, 

In [15]:
sess = tf.Session()

In [16]:
saver = tf.train.import_meta_graph('./models/model1937_gpu/model1937.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/model1937_gpu/'))

INFO:tensorflow:Restoring parameters from ./models/model1937_gpu/model1937.ckpt


In [17]:
graph = tf.get_default_graph()

In [18]:
X = graph.get_tensor_by_name("input/input_es:0")
#y = graph.get_tensor_by_name("input/target_na:0")

In [19]:
output_NN = graph.get_tensor_by_name("output/xw_plus_b:0")
#output_NN = graph.get_tensor_by_name("output_1:0")

In [20]:
feed_dict = {X:test_vectors}


In [21]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (150, 128)


In [22]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]

In [23]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]

In [24]:
es[es[0]=="adquisición"].index

Int64Index([2222], dtype='int64')

In [25]:
na[na[0]=="v"].index

Int64Index([1376], dtype='int64')

In [26]:
eval_es[6]

'nido'

In [27]:
closest[6]

['nepapan',
 'motemilih',
 'mota',
 'huipan',
 'tenanco',
 'tlazo',
 'diego',
 'cuexan',
 'dral',
 'secretario']

In [28]:
resultados = {palabra_es:top_10_nah for (palabra_es,top_10_nah) in zip(eval_es,closest)}

In [29]:
resultados

{'acabar': ['tlami',
  'mic',
  'cauh',
  'yoltequipacho',
  'auh',
  'ahui',
  'miqui',
  'yauh',
  'zque',
  'im'],
 'admirar': ['mahuizo',
  'imottilih',
  'tlamahuiz',
  'itt',
  'xip',
  'chililiz',
  'onimitz',
  'mahui',
  'zcuco',
  'oquich'],
 'adornar': ['uan',
  'chihchihua',
  'apana',
  'tizque',
  'chicahua',
  'quimon',
  'cequin',
  'yahualoa',
  'puchteca',
  'chiuh'],
 'adquisición': ['chiquiuh',
  'oticma',
  'mel',
  'amech',
  'tzotzona',
  'tlatzotzonque',
  'lahtol',
  'amaneapan',
  'quipiaz',
  'tzaoctimanca'],
 'agregar': ['motlali',
  'tiliaya',
  'tlahuicxitilli',
  'cuacualaca',
  'itzoltilia',
  'tlalia',
  'huic',
  'tec',
  'cuacua',
  'iyauh'],
 'aguacate': ['aztlacapalli',
  'tolachno',
  'ahua',
  'pix',
  'poyox',
  'diego',
  'yb',
  'dral',
  'rosario',
  'tzetzeloa'],
 'aire': ['yehyecatl',
  'eheca',
  'tetzahui',
  'eca',
  'hui',
  'yeyeca',
  'mach',
  'xa',
  'xip',
  'tlacochcalca'],
 'alegrar': ['papa',
  'tlaza',
  'coco',
  'pahpaqui',
  

In [30]:
esp = list(eval_set["esp"].values)

In [31]:
nah = list(eval_set["nah"].values)

In [32]:
pares_eval = list(zip(esp,nah))

In [33]:
from collections import defaultdict

In [34]:
gold = defaultdict(list)
for palabra_es,palabra_na in pares_eval:
    gold[palabra_es].append(palabra_na)


In [35]:
gold = dict(gold)

In [36]:
gold

{'acabar': ['tlami', 'tami'],
 'admirar': ['mahuizo', 'iglesia', 'tlamahuiz'],
 'adornar': ['apana', 'chihchihua'],
 'adquisición': ['quipiaz', 'tocococauh', 'cococauh', 'cococa'],
 'agregar': ['motlali', 'tlalia'],
 'aguacate': ['ahua'],
 'aire': ['yehyecatl', 'yeyeca', 'eheca', 'eheca'],
 'alegrar': ['ahuia', 'pahpa', 'pahpaqui'],
 'alfredo': ['alfredo'],
 'alimento': ['pal', 'atetzocoa'],
 'amado': ['tlazo', 'tlazohtla'],
 'amargo': ['chichi'],
 'animal': ['yolca', 'yoyolcameh'],
 'anáhuac': ['nahuac'],
 'barca': ['acalli'],
 'brazo': ['aca'],
 'caja': ['petlacal'],
 'cargo': ['tlahtoca', 'toca'],
 'centenario': ['nario', 'presi'],
 'chico': ['notelpochtli', 'tzitz', 'tepi'],
 'chiquillo': ['pipil'],
 'cierto': ['melahua', 'nel', 'cecni'],
 'ciudad': ['altpe', 'altepe'],
 'cofre': ['top', 'petlacal'],
 'coger': ['ana', 'cana', 'cui'],
 'comenzar': ['pehua', 'peuh'],
 'comer': ['cuaz', 'cua', 'tlacual', 'otlacua', 'cualtiz'],
 'conocer': ['ixmati', 'ixmachilia', 'mati', 'oixmatilia',

In [37]:
gold["amado"]

['tlazo', 'tlazohtla']

In [38]:
resultados["amado"]

['tlazohtla',
 'ama',
 'zquia',
 'ticcua',
 'tazohta',
 'diego',
 'tlazo',
 'comal',
 'niz',
 'federal']

In [39]:
resultados["aguacate"][1:5]

['tolachno', 'ahua', 'pix', 'poyox']

In [40]:
gold["comer"][1] in resultados["comer"][0]

True

In [41]:
resultados["aguacate"][0:1]

['aztlacapalli']

In [42]:
list(resultados.keys())

['ver',
 'efecto',
 'alfredo',
 'pipián',
 'enorme',
 'vaca',
 'nido',
 'niño',
 'conocer',
 'sólo',
 'caja',
 'vasija',
 'acabar',
 'fino',
 'final',
 'alegrar',
 'adornar',
 'coyote',
 'comer',
 'diverso',
 'adquisición',
 'verde',
 'llanura',
 'venir',
 'patio',
 'vender',
 'teatro',
 'echar',
 'muñeca',
 'tepaneca',
 'pieza',
 'indio',
 'derribar',
 'vestido',
 'pasar',
 'establecer',
 'tomar',
 'quitar',
 'chiquillo',
 'portal',
 'aguacate',
 'galleta',
 'jadear',
 'llevar',
 'barca',
 'puramente',
 'cultura',
 'seno',
 'ocurrir',
 'desierto',
 'ropa',
 'esfuerzo',
 'dar',
 'cargo',
 'tapar',
 'rogar',
 'nombre',
 'levantar',
 'tortuga',
 'alimento',
 'aire',
 'quetzal',
 'horno',
 'nabo',
 'descanso',
 'ciudad',
 'amargo',
 'animal',
 'muerto',
 'sueño',
 'destrozar',
 'pedirle',
 'mirada',
 'tigre',
 'ramo',
 'presidente',
 'familiar',
 'paso',
 'yo',
 'pez',
 'izquierda',
 'tráiganla',
 'presencia',
 'uña',
 'gritar',
 'volver',
 'indígena',
 'desear',
 'amado',
 'quiltoniles',

In [43]:
len(list(resultados.keys()))

150

In [44]:
p1 = 0
p5 = 0
p10 = 0
f1,f5,f10=False,False,False
palabra_gold = "madera"
hits=list()

not_found = 0

for palabra_gold in list(resultados.keys()):
    for i in gold[palabra_gold]:
        if i in resultados[palabra_gold]:
            hits.append(resultados[palabra_gold].index(i))
    if hits.__len__() > 0:
        if min(hits) == 0:
            p1 += 1
            p5 += 1
            p10 += 1
        if min(hits) >= 1 and min(hits) <= 5:
            p5 += 1
            p10 += 1
        if min(hits) > 5 and min(hits) <= 10:
            p10 += 1
        print(palabra_gold,min(hits),hits,p1,p5,p10)
    else:
        not_found += 1
        print(palabra_gold+": NOT FOUND")
    
    
    
        
    hits.clear()

print("\nP@1:",p1/len(list(resultados.keys())),"\tP@5:",p5/len(list(resultados.keys())),
      "\tP@10:",p10/len(list(resultados.keys())))



ver 0 [1, 0] 1 1 1
efecto: NOT FOUND
alfredo 0 [0] 2 2 2
pipián 0 [0] 3 3 3
enorme: NOT FOUND
vaca 0 [0, 4] 4 4 4
nido: NOT FOUND
niño 1 [2, 5, 6, 1] 4 5 5
conocer 0 [0, 7, 2, 4] 5 6 6
sólo 0 [0] 6 7 7
caja 0 [0] 7 8 8
vasija 0 [0, 6, 1] 8 9 9
acabar 0 [0] 9 10 10
fino 1 [1, 2] 9 11 11
final: NOT FOUND
alegrar 3 [8, 3] 9 12 12
adornar 1 [2, 1] 9 13 13
coyote 1 [1] 9 14 14
comer 0 [1, 0, 3, 4] 10 15 15
diverso 0 [0] 11 16 16
adquisición 8 [8] 11 16 17
verde 0 [0, 1] 12 17 18
llanura 0 [0] 13 18 19
venir 0 [1, 0] 14 19 20
patio 0 [1, 0, 2] 15 20 21
vender 0 [1, 0, 3, 2] 16 21 22
teatro 0 [0, 2] 17 22 23
echar 1 [1] 17 23 24
muñeca 0 [0] 18 24 25
tepaneca 1 [1] 18 25 26
pieza 0 [0] 19 26 27
indio 1 [1] 19 27 28
derribar 2 [2] 19 28 29
vestido 0 [8, 0] 20 29 30
pasar 0 [0] 21 30 31
establecer: NOT FOUND
tomar 2 [2, 7] 21 31 32
quitar 4 [4] 21 32 33
chiquillo: NOT FOUND
portal 2 [2, 5] 21 33 34
aguacate 2 [2] 21 34 35
galleta 0 [0] 22 35 36
jadear 0 [0] 23 36 37
llevar 0 [0, 4, 2] 24 37 38


In [45]:
not_found

16